# PA2.1 - Building your first Chatbot: Aleeza

### Introduction

In this notebook, you will be implementing your own version of the first ever Chatbot, ELIZA.

### Instructions

- Follow along with the notebook, filling out the necessary code where instructed.

- <span style="color: red;">Read the Submission Instructions and Plagiarism Policy in the attached PDF.</span>

- <span style="color: red;">Make sure to run all cells for credit.</span>

- <span style="color: red;">Do not remove any pre-written code.</span> We will be using the `print` statements to grade your assignment.

- <span style="color: red;">You must attempt all parts.</span> Do not assume that because something is for 0 marks, you can leave it - it will definitely be used in later parts.

## Background: ELIZA
#### The first ever Chatbot.

<div style="display: flex; justify-content: center;">
<img src="https://upload.wikimedia.org/wikipedia/commons/7/79/ELIZA_conversation.png" width="400" height="250" alt="ELIZA Conversation">
</div>

This Chatbot was released in 1966 (before the term Chatbot even existed) by Joseph Weizenbaum of MIT. This was a revolutionary program that allowed humans to converse with a computer. As you may have predicted, this was a retrieval-based system, very different from the "language models" of today. However, you may be surprised by how well it performed despite the simplicity of the system. Here is part of a conversation someone had with ELIZA, from the original paper (the capitalised sentences are ELIZA's responses):
\
\
Men are all alike.\
IN WHAT WAY\
They're always bugging us about something or other.\
CAN YOU THINK OF A SPECIFIC EXAMPLE\
Well, my boyfriend made me come here.\
YOUR BOYFRIEND MADE YOU COME HERE\
He says i'm depressed much of the time.\
I AM SORRY TO HEAR YOU ARE DEPRESSED\
It's true. I am unhappy.\
DO YOU THINK COMING HERE WILL HELP YOU NOT TO BE UNHAPPY\
......

The program used certain programmed rules to "transform" the input into the output. In order to do this, the program must first decompose the sentence based on certain criteria and then reassemble it based on the predefined assembly specifications. For example, if it is provided with the input sentence, "It seems that you hate me", it may be decomposed into:

1) It seems that
2) you
3) hate
4) me

Of these, (2) and (4) are recognised as key words. The program can then use the remaining sections of the sentence based on pre-defined rules to construct an output. For example, it may be programmed with the rule:

decomposition template:\
(0 YOU 0 ME)\
and the reassembly rule:\
(WHAT MAKES YOU THINK I 3 YOU).

Here, the "0" represents any number of words, whereas the "3" represents the 3rd part of the sentence from before. Hopefully, this makes the implementation a little clearer. If not, don't worry as you'll understand how it works once you start implementing your own version!

For more details on the original ELIZA implementation, [Click Here](https://web.stanford.edu/class/cs124/p36-weizenabaum.pdf).


## Specifications

As described above, your task will be to first read in a user string, then modify it to provide an output (sometimes subtly, sometimes drastically, depending on the input string). This should be easy to do with the regex library, the specifics of which were discussed in class.

\
Your program should be able to handle all 1st and 2nd person pronouns, all 1st and 2nd person subject-verb pairs with the verb be and all possible forms of the verb. If it is unclear what is meant by this, you might want to do some googling.

\
An example is as follows:

Regular Expression: I am (.*)\
Response: How long have you been %1?

Example Input that matches: I am sad.\
Example Response: How long have you been sad?

Please note that this is a simplified version of the chatbot, and the original bot had a much more complex algorithm behind it.

You will have two tables to store all the logic of your bot:
1. Reflection Table
2. Response Table

These will be described in detail in the cells below.

## Imports

These are the ONLY imports you can use for this part of the assignment.

In [53]:
import json
import re
import random

## Tables

These are your reflection and response tables.

#### Reflection Table

This table serves to convert your pronouns from first person to second person and vice versa. You should list all forms of the pronouns and their corresponding "reflection". (eg. i : you)\
\
You should also do the same for all the forms of the verb "be". (eg. am : are)\
\
Note: You do not need to add plural pronouns such as "we".\
\
This table will be represented as a dictionary. (The first entry is listed as an example below)

In [54]:
reflectionTable = {
    "i"    : "you",
    "me": "you",
    "my": "your",
    "am": "are",
    # Add entries here

    #forms of verb be
    "is": "are",
    "was": "were",
    "were": "was",
    "being": "being",
    "been": "been",
    "be": "be"
}

#### Response Table

This table is in the form of a nested list. Each entry is a list, with the first term being your regular expression and the second term being a list of possible responses. "%n" represents the nth match. You will need to handle this in your code later when replacing the relevant parts of the text.

Since this is a fairly large table, you will fill out the regular expressions and the responses in a json file: "responseTable.json"

\
In this table, you must include ALL subject-verb pairs for the verb "be". Do this for first, second and third person pronouns. (eg. I am ...) You must add at least 3 appropriate responses for each of these pairs. You need not account for the contracted versions of the pairs. But, DO include the corresponding question statements for each of these pairs. You can assume there will be no past-tense or future-tense inputs.\
\
Furthermore, in the case that you encounter no matches, you must have fallbacks. Due to this, you must also account for the following cases:
1. (I feel ...), (I want ...), (I think ...)
2. Subject with an unknown verb
3. An unrecognised question
4. Any string

Include 4 or more responses for these cases as they will likely be encountered more often.\
\
Lastly, add at least 3 more subject-verb pairs, with at least 1 response each. These can be anything you like. Have fun with it (but keep it appropriate).\
\
For example:

Regex: I voted for (.*)

Response: How did voting for (.*) make you feel?

Please ensure the correct order, as you will only be checking the first match later on.\
Once again, an example entry has been provided.

In [55]:
# Add entries in the JSON file

responseTable = json.load(open('responseTable.json'))
#print(responseTable[0][0])

## Helper Functions (Optional)

If you wish to modularise your code to make your life simpler in the upcoming cells. Please define your helper functions here.

## Aleeza Class

This is the class you will be implementing all of your bot's functionality in. As you will see, this is very straightforward and most of the actual work will be done while writing the response table. We will call our version Aleeza.

In [56]:
class Aleeza:
  def __init__(self, reflectionTable, responseTable):
    """
    Initiliase your bot by storing both the tables as instance variables.
    You can store them any way you want. (Dictionary, List, etc.)
    """
    self.__reflectionTable=reflectionTable #This is a dictionary
    self.__responseTable=responseTable

    # Code here

  def reflect(self, text):
    """
    Take a string and "reflect" based on the reflectionTable.

    Return the modified string.
    """
    
    # Code here
    text=text.lower()
    new_text=text
    text=text.split()
    for word in text:
      if word in self.__reflectionTable:
        new_word=self.__reflectionTable[word]
        new_text=new_text.replace(word,new_word)

    return new_text

  def respond(self, text):
    """
      Take a string, find a match, and return a randomly
      chosen response from the corresponding list.

      Do not forget to "reflect" appropriate parts of the string.

      If there is no match, return None.
    """

    # Code here
    #Converting text to lower case
    text=text.lower()
    for match_pattern, responses in self.__responseTable:
      #Regex is case sensitive
      match_pattern=match_pattern.lower()
      match_found=re.match(match_pattern,text)
      if match_found:
        #Finding the sentence using the capture group
        captured_sentence=match_found.group(1)
        #reflecting the sentence
        reflected_sentence=self.reflect(captured_sentence)
        #Picking up a random response from the responses and then reflecting it
        random_response=random.choice(responses)
        response=random_response.replace('%1',reflected_sentence)
        return response
      
    return None

## Test your Bot

You can use this interface to manually check your bot's responses.

In [57]:
def command_interface():
    print('Aleeza\n---------')
    print('Talk to the program by typing in plain English.')
    print('='*72)
    print('Hello.  How are you feeling today?')

    s = ''
    therapist = Aleeza(reflectionTable, responseTable)
    while s != 'quit':
        try:
            s = input('> ')
        except EOFError:
            s = 'quit'
        print(s)
        while s[-1] in '!.':
            s = s[:-1]
        print(therapist.respond(s))

In [58]:
command_interface()

Aleeza
---------
Talk to the program by typing in plain English.
Hello.  How are you feeling today?
i am sad
You are sad?
i want to be the best
How would that make you feel?
i want to play
Why do you want to play?
i believe i can fly
Why do you believe you can fly?
hello?
Could you please elaborate?
hi there
I'm sorry. I don't understand
he is angry
How long has he been angry?



IndexError: string index out of range

## Test Sentences

After testing your bot, you have likely seen that it does not work very well yet. This goes to show the immense amount of work that was put into the original ELIZA program.\
In any case, having concocted all of your (hopefully) appropriate responses, you now need to demonstrate your bot handling all the cases listed above. To do this, you must provide an example sentence handling each of the regular expressions you have listed in your response table.

In [69]:
#Cases considered:
#1. "I am (.*)" -> anything followed by I am ____.
#2. I want|feel|think (.*) -> anything followed by I want/feel/think _____.
#3. ".*\?" -> any thing with a question mark that doesn't have a response
#4. ".*" -> any string, considering cases where we need to respond somethingn and keep convo going.
#5. 3 subject verb pairs - I enjoy, I hate, I believe
#6 He/she/it only done with is i.e for be verb form
test_sentences = [
    # Add test sentences here
    "I am sad",
    "I want to be the best",
    "I feel that my mom loves me",
    "Today is a good day",
    "Can you help me get motivation?",
    "I think GenAI course is really fascinating",
    "I believe I can fly",
    "I hate politics",
    "I enjoy playing cricket",
    "He is really angry"

]

In [70]:
def get_responses(sentence_list, bot):
    """
    Get a response for each sentence from the list and return as a list.
    """

    # Code here
    response_list=[]
    for sentence in sentence_list:
        response_list.append(bot.respond(sentence))
    return response_list

In [71]:
therapist = Aleeza(reflectionTable, responseTable)

for pair in zip(test_sentences, get_responses(test_sentences, therapist)):
    print('='*72)
    print(pair[0])
    print(pair[1])

I am sad
What makes you sad?
I want to be the best
Why do you want to be the best?
I feel that my mom loves me
Why do you feel that your mom loves you?
Today is a good day
Could you please elaborate?
Can you help me get motivation?
I'm sorry, I don't understand the question.
I think GenAI course is really fascinating
Why do you think genai course are really fascinating?
I believe I can fly
How long have you been believing you can fly?
I hate politics
Why do you hate politics?
I enjoy playing cricket
How do you know that you enjoy playing cricket?
He is really angry
What makes him really angry?


# Giving Aleeza Emotional Intelligence

In the next part of the assignment, you will be giving your chatbot some emotional intelligence. This will be done by training a simple emotion classification model. You will then use this model to classify the sentiment of the user's input and respond accordingly.\
\
How our logic will work is as follows:
1. If there is a match in the response table, we will use the response from the table.
2. If there is no match, we will classify the emotion of the input and respond accordingly.

The model we will use is a simple Naive Bayes Classifier. This is a simple model that works well with text data. You will be using the `scikit-learn` library to train the model, and the huggingface `datasets` library to get the data.

## Imports

These are the ONLY imports you can use for this part of the assignment.

In [72]:
%pip install datasets scikit-learn
import datasets
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Dataset

We will be using the `emotion` dataset from the `datasets` library. This dataset contains text data and the corresponding emotion. You will use this data to train your model. Load this dataset using the `load_dataset` function from the `datasets` library.

Next, split the dataset into training and testings sets.\
(HINT: This has already been done for you in the dataset you loaded)

In [74]:
"""
Load the emotion dataset from Hugging Face
"""
from datasets import load_dataset
dataset =load_dataset('emotion')

# print(dataset)

In [75]:
"""
Split the dataset into training and testing sets
"""
# print(dataset['train']['text'])
# print(dataset['train']['label'])

# Code below
train_data = dataset['train']['text']
train_labels = dataset['train']['label']
test_data = dataset['test']['text']
test_labels = dataset['test']['label']

## Training the Model

Just like in your previous assignment, you will now train the model and evaluate it.

In [76]:
"""
Vectorise the data and train the model
"""

#Initialising
vectorizer=CountVectorizer()
clf=MultinomialNB()

#Vectorising the data
#The vectoriser is only fit onto the training data and not on the test data so we will fit_transform on training data
#but only transform the test datta
vectorized_training_data=vectorizer.fit_transform(train_data)
vectorized_test_data=vectorizer.transform(test_data)

#Training the model
clf.fit(vectorized_training_data,train_labels)


"""
Predict on the test set
"""

predicted_labels = clf.predict(vectorized_test_data)


"""
Print classification report
"""
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

           0       0.74      0.94      0.83       581
           1       0.74      0.97      0.84       695
           2       0.95      0.23      0.37       159
           3       0.92      0.57      0.70       275
           4       0.82      0.53      0.64       224
           5       0.00      0.00      0.00        66

    accuracy                           0.77      2000
   macro avg       0.69      0.54      0.56      2000
weighted avg       0.77      0.77      0.73      2000



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

## Putting it all together

Now that we have our classification model, we can modify our chatbot to use it.

First, we will remove the fallback responses from our response table, i.e. the following cases:
1. (I feel ...), (I want ...), (I think ...)
2. Subject with an unknown verb
3. An unrecognised question
4. Any string

Remove these and save your response table as "responseTable2.json".

In [77]:
# Make a new file "responseTable2.json" and add your modified table to it

responseTable = json.load(open('responseTable2.json'))

#### Emotion Response Table

This table will be a dictionary with the emotions as keys and a list of possible responses as values. You should include at least 2 responses for each emotion.

In [78]:
# Add responses below

emotionTable = {
    0 : [ # sadness
        "I am sorry to hear that.",
        "It will be alright. Don't worry!"
    ], 
    1 : [ # joy
        "I am so happy for you.",
        "Well, that sounds amazing!!"
    ],
    2 : [ # love
        "You seem to have fallen in love with this.",
        "You really love this!"
    ],
    3 : [ # anger
        "It's okay. You need to calm down first.",
        "Get a glass of water. You seem angry."
    ],

    4 : [ # fear
        "Don't worry. You got this!",
        "There is nothing to be scared about. Just relax!"
    ],

    5 : [ # surprise
        "WOW!",
        "I AM SURPRISED!"
    ]
}



### Modifying your Chatbot

You will now modify your chatbot to use the emotion classifier. If there is a match in the response table, we will use the response from the table. If there is no match, we will classify the emotion of the input and respond accordingly.

In [79]:
class IntelligentAleeza(Aleeza):
    def __init__(self, reflectionTable, responseTable, emotionTable, classifier):
        """
        Initialise your bot by calling the parent class's __init__ method,
        and then storing the emotionTable as an instance variable.

        Next, store the classification model as an instance variable.
        """
        
        # Code here
        #Calling constructor of parent class
        Aleeza.__init__(self,reflectionTable,responseTable)
        self.__emotionTable=emotionTable
        self.__classifer=classifier

    def smart_respond(self, text):
        """
        Take a string, call the parent class's respond method.
        If the response is None, then respond based on the emotion.
        """

        # Code here
        response_from_parent= self.respond(text)
        if response_from_parent!=None:
            return response_from_parent
        else:
            vectorized_sentence=vectorizer.transform([text])
            emotion=self.__classifer.predict(vectorized_sentence)
            response_acc_to_emotion=random.choice(self.__emotionTable[emotion[0]])
            return response_acc_to_emotion
            
            
            

## Test your New Bot

Randomly select 5 sentences from the test set and test your bot. You should see that it now responds with an appropriate message based on the emotion detected in the input (when there is no match).

In [80]:
def get_responses(sentence_list, bot):
    """
    Get a response for each sentence from the list and return as a list.
    Use your new smart_respond method.
    """
    response_list=[]
    for sentence in sentence_list:
        response_list.append(bot.smart_respond(sentence))
    return response_list
        

In [82]:
"""
Create an instance of the IntelligentAleeza class
"""
intelligent_therapist = IntelligentAleeza(reflectionTable,responseTable,emotionTable,clf)

"""
Get 5 random test instances from the test data
"""

# Code here
test_instances = []
for i in range(5):
    random_index=random.randint(0, 1999)
    test_instances.append(test_data[random_index])


""" 
Get responses from the intelligent_therapist 
"""

responses = get_responses(test_instances, intelligent_therapist)


"""
Print the test instances and the responses
"""
for pair in zip(test_instances, responses):
    print('='*72)
    print(pair[0])
    print(pair[1])

i exist for does my existence even mean anything to anyone apart from my family i always wonder about my existence and the fuck now i feel so dumb ive never thought about the purpose of it
I am sorry to hear that.
i feel is he generous
I am so happy for you.
i feel his hand on me to stay faithful
Well, that sounds amazing!!
i feel as though most people will find it quite pleasant
Well, that sounds amazing!!
i still feel more than anything else humiliated whenever i think of everything that s happened
I am sorry to hear that.


# Fin.